In [2]:
import pandas as pd
import collections

df = pd.read_csv('MaterialTransactionsExport1.csv', dtype='unicode')
pd.set_option("display.max_rows", None, "display.max_columns", None)

df['Date and Time of Trans'] = pd.to_datetime(df['Date and Time of Trans'])
df['Quantity'] = df['Quantity'].str.replace(',','').astype(float)

wclist = ['COST','SANDER','GL-H','EDGE','GL-C2','GL-P2','GL-C3','GL-P3','SAWS','TEKNA','ROUTER','PACK']
badwclist = ['OSP','SHEET','INJASY','INJFAS','SBLEND','SHEAR', 'PRESS']
famlist = ['NUDO','PRIME','TRUCKDOOR']
prohib = ['EDR','END','EPS','EXM','EXR','EXT','ICUBE']

new = df[df['Family Code'].isin(famlist)]
new = new[~new['Prod Code'].isin(prohib)]
new = new[new['Quantity'].astype(int) > 0]
new = new[new['Work Center'].notna()]

filt = new[['Item','Quantity','Work Center','Date and Time of Trans']]


#g = filt.groupby(['Item','Work Center','Quantity','Date and Time of Trans'])['Quantity'].count()
g = filt.sort_values('Item')

d = {}

for i in g['Item'].unique():
    d[i] = [{g['Work Center'][j]: g['Quantity'][j]} for j in g[g['Item']==i].index]

for item, ops in d.copy().items():
    wc = []
    for pair in ops:
        wc.append(list(pair.keys())[0])
    if 'COST' not in wc:
        d.pop(item)

for item, ops in d.copy().items():
    wc = []
    for pair in ops:
        wc.append(list(pair.keys())[0])
    check = any(item in wc for item in badwclist)
    if check == True:
        d.pop(item)

#for key, val in d.items():
    #print(key)
    #print(val)

seq = []
items = []
for item, ops in d.items():
    items.append(str(item))
    counter = collections.Counter()
    for pair in ops:
        counter.update(pair)
    seq.append(dict(counter))
print(seq)
flows = dict(zip(items,seq))
print(len(flows.keys()))

[{'PACK': 222.0, 'SAWS': 222.0, 'COST': 222.0}, {'SANDER': 2645.0, 'COST': 2645.0, 'PACK': 2637.0, 'SAWS': 2637.0, 'GL-P2': 2645.0, 'GL-C2': 2645.0}, {'EDGE': 120.0, 'COST': 60.0, 'GL-H': 60.0, 'GL-P2': 60.0, 'PACK': 60.0, 'GL-C2': 60.0}, {'GL-C2': 53.0, 'PACK': 53.0, 'SAWS': 53.0, 'GL-P2': 53.0, 'SANDER': 53.0, 'COST': 53.0}, {'GL-P2': 282.0, 'EDGE': 458.0, 'PACK': 282.0, 'COST': 282.0, 'GL-H': 282.0, 'GL-C2': 282.0, 'SAWS': 53.0}, {'PACK': 88.0, 'SAWS': 88.0, 'GL-C2': 88.0, 'GL-P2': 88.0, 'GL-H': 88.0, 'SANDER': 88.0, 'COST': 88.0}, {'ROUTER': 1.0, 'COST': 1.0, 'SAWS': 1.0, 'PACK': 1.0}, {'COST': 116.0, 'SAWS': 116.0, 'TEKNA': 1.0, 'PACK': 1.0}, {'PACK': 20.0, 'SAWS': 20.0, 'Q-SAWS': 20.0, 'GL-SPR': 20.0, 'GL-H': 20.0, 'COST': 20.0}, {'COST': 22.0, 'GL-H': 22.0, 'GL-SPR': 22.0, 'Q-SAWS': 22.0, 'SAWS': 22.0, 'PACK': 22.0}, {'Q-SAWS': 324.0, 'COST': 324.0, 'GL-H': 324.0, 'GL-SPR': 324.0, 'SAWS': 324.0, 'PACK': 324.0}, {'Q-SAWS': 35.0, 'SAWS': 35.0, 'PACK': 35.0, 'GL-H': 35.0, 'GL-SPR':

In [3]:
flow = pd.DataFrame(flows)
flow = flow.reindex(['COST','SANDER','GL-H','EDGE','GL-C2','GL-P2','GL-C3','GL-P3','GL-SPR','SAWS','TEKNA','ROUTER','PACK'])#,'Q-SAWS','QC','Q-GL','Q-FAI','FILM','SCORE','FRAME','HANDRT','FIRETR','PAINT','IFAB','SCRAPE','EXPAND'])
flow = pd.DataFrame.transpose(flow)
print(flow)

                                   COST  SANDER     GL-H     EDGE   GL-C2  \
10M 48.5"X72.125" WHITE FLUTED    222.0     NaN      NaN      NaN     NaN   
A3P375-8-BLACK                   2645.0  2645.0      NaN      NaN  2645.0   
A3P500-8-ME2-BLACK                 60.0     NaN     60.0    120.0    60.0   
A3P625-8-BLACK                     53.0    53.0      NaN      NaN    53.0   
A3P625-8-ME2-BLACK                282.0     NaN    282.0    458.0   282.0   
A3P750-8-ME2-BLACK                 88.0    88.0     88.0      NaN    88.0   
AFFILIATED-PO57510-20-QE10P750      1.0     NaN      NaN      NaN     NaN   
AFFILIATED-PO58286-20-D-J-M-N     116.0     NaN      NaN      NaN     NaN   
APM075-10-H2P-PF-BEIGEWOOD         20.0     NaN     20.0      NaN     NaN   
APM075-10-H2P-PF-BEIGEWOOD XG      22.0     NaN     22.0      NaN     NaN   
APM075-10-H2P-PF-CITADEL WARP     324.0     NaN    324.0      NaN     NaN   
APM075-10-H2P-PF-FASHION GREY      35.0     NaN     35.0      NaN     NaN   

In [7]:
#flow.to_csv('output.csv', encoding='utf-8', index=True)

#FOR EDGE:
#BEFORE GL: both a hm op and a cold glue op
#BEFORE AND AFTER on GL: two sided only 
#AFTER GL: only 1 gl op

#FOR SAWS:


#list all possible wc pairs, count across all items and sum by item quant
pairs = [(a,b) for idx, a in enumerate(wclist) for b in wclist[idx + 1:]]
print(pairs)

[('COST', 'SANDER'), ('COST', 'GL-H'), ('COST', 'EDGE'), ('COST', 'GL-C2'), ('COST', 'GL-P2'), ('COST', 'GL-C3'), ('COST', 'GL-P3'), ('COST', 'SAWS'), ('COST', 'TEKNA'), ('COST', 'ROUTER'), ('COST', 'PACK'), ('SANDER', 'GL-H'), ('SANDER', 'EDGE'), ('SANDER', 'GL-C2'), ('SANDER', 'GL-P2'), ('SANDER', 'GL-C3'), ('SANDER', 'GL-P3'), ('SANDER', 'SAWS'), ('SANDER', 'TEKNA'), ('SANDER', 'ROUTER'), ('SANDER', 'PACK'), ('GL-H', 'EDGE'), ('GL-H', 'GL-C2'), ('GL-H', 'GL-P2'), ('GL-H', 'GL-C3'), ('GL-H', 'GL-P3'), ('GL-H', 'SAWS'), ('GL-H', 'TEKNA'), ('GL-H', 'ROUTER'), ('GL-H', 'PACK'), ('EDGE', 'GL-C2'), ('EDGE', 'GL-P2'), ('EDGE', 'GL-C3'), ('EDGE', 'GL-P3'), ('EDGE', 'SAWS'), ('EDGE', 'TEKNA'), ('EDGE', 'ROUTER'), ('EDGE', 'PACK'), ('GL-C2', 'GL-P2'), ('GL-C2', 'GL-C3'), ('GL-C2', 'GL-P3'), ('GL-C2', 'SAWS'), ('GL-C2', 'TEKNA'), ('GL-C2', 'ROUTER'), ('GL-C2', 'PACK'), ('GL-P2', 'GL-C3'), ('GL-P2', 'GL-P3'), ('GL-P2', 'SAWS'), ('GL-P2', 'TEKNA'), ('GL-P2', 'ROUTER'), ('GL-P2', 'PACK'), ('GL-C3

In [5]:
if sander and gl-h in seq

SyntaxError: invalid syntax (<ipython-input-5-fbf7ae4392da>, line 1)